In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langsmith import Client

In [3]:
# LangSmith 설정
client = Client()
datasets = client.list_datasets(dataset_name="test1")
try:
    dataset = next(iter(datasets))
except StopIteration:
    print("No Dataset found")

In [4]:
# LangSmith 설정
client = Client()
for feed in client.list_feedback():
    print(feed)
    break

id=UUID('599e5028-4219-47d3-9c35-df244adfce8a') created_at=datetime.datetime(2024, 3, 30, 6, 10, 5, 484332) modified_at=datetime.datetime(2024, 3, 30, 6, 10, 6, 167017) run_id=UUID('56e05ab2-cad6-4654-b3b8-9cf380779a2c') key='thumbs' score=1.0 value=None comment='대한민국의 수도는 어디야?' correction=None feedback_source=FeedbackSourceBase(type='api', metadata={}) session_id=UUID('c8f964ce-5210-467b-ba83-8f8d3bf5e716')


In [5]:
for k, v in client.read_run(feed.run_id):
    if k == "outputs":
        print(v)
    if k == "extra":
        print(v)
    print(k, v)

id 56e05ab2-cad6-4654-b3b8-9cf380779a2c
name RunnableWithMessageHistory
start_time 2024-03-30 06:09:48.653877
run_type chain
end_time 2024-03-30 06:09:49.490328
{'metadata': {'query': '대한민국의 수도는 어디야?', 'session_id': 'C09633'}}
extra {'metadata': {'query': '대한민국의 수도는 어디야?', 'session_id': 'C09633'}}
error None
serialized {'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'], 'kwargs': {'first': {'id': ['langchain', 'schema', 'runnable', 'RunnableBinding'], 'kwargs': {'bound': {'id': ['langchain', 'schema', 'runnable', 'RunnableAssign'], 'kwargs': {'mapper': {'id': ['langchain', 'schema', 'runnable', 'RunnableParallel'], 'kwargs': {'steps': {'history': {'id': ['langchain', 'schema', 'runnable', 'RunnableBinding'], 'kwargs': {'bound': {'id': ['langchain_core', 'runnables', 'base', 'RunnableLambda'], 'lc': 1, 'repr': 'RunnableLambda(_enter_history)', 'type': 'not_implemented'}, 'config': {'run_name': 'load_history'}, 'config_factories': [], 'custom_input_type': None, 'custom_output

In [6]:
def extract_qa_from_runid(run_id, session_id=None):
    ret = dict()
    filter_flag = True
    try:
        for k, v in client.read_run(run_id):
            if k == "outputs":
                ret["answer"] = v["output"]["content"]

            if k == "extra":
                ret["question"] = v["metadata"]["query"]
                if session_id is not None:
                    sid = v["metadata"]["session_id"]
                    if sid.strip() == session_id.strip():
                        ret["session_id"] = sid.strip()
                    else:
                        filter_flag = False
                elif "session_id" in v["metadata"]:
                    ret["session_id"] = v["metadata"]["session_id"].strip()
                else:
                    ret["session_id"] = ""
        if filter_flag:
            return ret
        else:
            return None
    except Exception as e:
        print(e)
        return None

In [7]:
extract_qa_from_runid(feed.run_id, session_id=None)

{'question': '대한민국의 수도는 어디야?', 'session_id': 'C09633', 'answer': '서울 🇰🇷'}

In [8]:
examples = client.list_examples(dataset_id=dataset.id)
for example in examples:
    print(example)
    break

dataset_id=UUID('f37aeb6c-aeaa-4b4c-b47f-8ad4c5667e29') inputs={'question': '답변에 적절한 이모지도 추가해줘'} outputs={'output': {'type': 'ai', 'content': '서울 🇰🇷', 'example': False, 'additional_kwargs': {}, 'response_metadata': {'finish_reason': 'stop'}}} metadata=None id=UUID('f1a0f23a-adc8-40ce-b8b7-9fa492f9ec9b') created_at=datetime.datetime(2024, 3, 30, 6, 11, 0, 841358, tzinfo=datetime.timezone.utc) modified_at=datetime.datetime(2024, 3, 30, 6, 11, 0, 841358, tzinfo=datetime.timezone.utc) runs=[] source_run_id=UUID('56e05ab2-cad6-4654-b3b8-9cf380779a2c')


In [9]:
def create_qa_pairs(dataset_id, session_id=None):
    qa_pairs = []
    examples = client.list_examples(dataset_id=dataset_id)
    for exp in examples:
        qa = dict()
        ret = extract_qa_from_runid(exp.source_run_id, session_id)

        if ret is None:
            continue
        qa["question"] = ret["question"]
        qa["answer"] = ret["answer"]
        qa["session_id"] = ret["session_id"]
        qa_pairs.append(qa)
    return qa_pairs

In [10]:
qa_pairs = create_qa_pairs(dataset.id, session_id=None)
qa_pairs

[{'question': '대한민국의 수도는 어디야?', 'answer': '서울 🇰🇷', 'session_id': 'C09633'}]

In [11]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [12]:
example_prompt = PromptTemplate(
    input_variables=["instruction", "question", "answer"],
    template="Question: {question}\nResponse: {answer}",
)

print(example_prompt.format(**qa_pairs[0]))

Question: 대한민국의 수도는 어디야?
Response: 서울 🇰🇷


In [13]:
prompt = FewShotPromptTemplate(
    examples=qa_pairs,
    example_prompt=example_prompt,
    suffix="----\nQuestion: {question}\nResponse: ",
    input_variables=["question"],
)

question = "중국의 수도는 어디인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question: 대한민국의 수도는 어디야?
Response: 서울 🇰🇷

----
Question: 중국의 수도는 어디인가요?
Response: 


In [14]:
final_prompt = PromptTemplate.from_template(
    """Please refer to the INSTRUCTIONS and FEW SHOT examples to answer the question.
    
#FEW EXAMPLES:
{few_shot}                             

#INSTUCTIONS:
{instruction}
"""
)

In [15]:
prompt_string = final_prompt.format(
    few_shot=prompt.format(question="프랑스의 수도는?"),
    instruction="한글로 간결하게 답변해 주세요",
)

print(prompt_string)

Please refer to the INSTRUCTIONS and FEW SHOT examples to answer the question.
    
#FEW EXAMPLES:
Question: 대한민국의 수도는 어디야?
Response: 서울 🇰🇷

----
Question: 프랑스의 수도는?
Response:                              

#INSTUCTIONS:
한글로 간결하게 답변해 주세요



In [16]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain_openai import ChatOpenAI


class StreamHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(token, end="", flush=True)

In [17]:
chain = final_prompt | ChatOpenAI(
    model="gpt-4-turbo-preview",
    temperature=0,
    streaming=True,
    callbacks=[StreamHandler()],
)

In [20]:
_ = chain.invoke(
    {
        "few_shot": prompt.format(question="프랑스의 수도는 어디야?"),
        "instruction": "한글로 간결하게 답변해 주세요",
    }
)

파리 🇫🇷